In [382]:
import os
import csv
import math
import pandas as pd
from numpy import genfromtxt
import numpy as np
import matplotlib.pyplot as plt
import math

actual_dist=25#varies with 1m to 25 m

# Set the path to the folder containing the dataset
# data_folder = 'C:\Users\LILA\Desktop\RTT-RSS with Kalman_offset\raw_dataset'
data_folder = r'C:\Users\LILA\Desktop\RTT-RSS with Kalman_offset\Validate_data'
#data_folder = 'C:/Users/LILA/RTT-RSS with Kalman/raw_dataset'
# Loop through each CSV file in the folder and read it into a DataFrame
for file_name in os.listdir(data_folder):
     if file_name == (str(actual_dist)+'m.csv'): #if it found filname after defining file path break
        file_path = os.path.join(data_folder, file_name)
        df = pd.read_csv(file_path)
        break
df.drop(['#<Time(ms)>','<Ch-MHz>','<Burst#>','<AP-SSID>','<RTT AP?>','<Successes#>'],axis=1,inplace=True)
df.rename(columns={'<True Range(m)>': 'True_Range', '<Est. Range(m)>': 'Est_Range','<RSSI(dBm)>': 'RSSI','<Std dev(m)>': 'stdev','<Successes#>': 'successes'}, inplace=True)
Est=df.Est_Range
R_RTT=Est.var()
data1_RTT=Est.to_list()
df

,True_Range,Est_Range,stdev,RSSI
0,25.0,27.092,0.542,-75
1,25.0,26.916,0.186,-73
2,25.0,26.819,4.349,-72
3,25.0,26.389,0.062,-80
4,25.0,25.920,0.286,-71
...,...,...,...,...
258,25.0,25.422,0.932,-65
259,25.0,25.832,0.279,-71
260,25.0,25.618,0.280,-71
261,25.0,25.774,0.305,-78


In [383]:
import numpy as np
import pandas as pd

def remove_outliers(df, col_name, threshold=1):
    """
    Remove outliers from a Pandas DataFrame using the Z-score method.
    
    Parameters:
        df (pandas.DataFrame): The DataFrame to remove outliers from.
        col_name (str): The column name to remove outliers from.
        threshold (float): The Z-score threshold for outlier detection. Default is 3.
        
    Returns:
        A new DataFrame with outliers removed.
    """
    # Calculate the Z-score for each value in the column
    z_scores = np.abs((df[col_name] - df[col_name].mean()) / df[col_name].std())
    
    # Create a boolean mask where values with a Z-score greater than the threshold are set to False
    mask = z_scores <= threshold
    
    # Return a new DataFrame with the outliers removed
    return df[mask]

# Load the original DataFrame with column name 'Est_Range'
col_name = 'Est_Range'

# Remove outliers from the 'Est_Range' column
df = remove_outliers(df, col_name, threshold=1)
df

,True_Range,Est_Range,stdev,RSSI
0,25.0,27.092,0.542,-75
1,25.0,26.916,0.186,-73
2,25.0,26.819,4.349,-72
3,25.0,26.389,0.062,-80
4,25.0,25.920,0.286,-71
...,...,...,...,...
258,25.0,25.422,0.932,-65
259,25.0,25.832,0.279,-71
260,25.0,25.618,0.280,-71
261,25.0,25.774,0.305,-78


In [384]:
offset=np.mean(df.Est_Range-df.True_Range)
offset

0.8795658914728677

In [385]:
# Apply the offset to the estimated range values to get the final range values
pd.options.mode.chained_assignment = None
df.loc[:,'Final_range'] = df['Est_Range'] - offset

In [386]:
# Calculate the error before applying the offset
df['Error_before_offset'] = abs(df['Est_Range'] - df['True_Range'])

# Calculate the error after applying the offset
df['Error_after_offset'] = df['Final_range'] - df['True_Range']
df

,True_Range,Est_Range,stdev,RSSI,Final_range,Error_before_offset,Error_after_offset
0,25.0,27.092,0.542,-75,26.212434,2.092,1.212434
1,25.0,26.916,0.186,-73,26.036434,1.916,1.036434
2,25.0,26.819,4.349,-72,25.939434,1.819,0.939434
3,25.0,26.389,0.062,-80,25.509434,1.389,0.509434
4,25.0,25.920,0.286,-71,25.040434,0.920,0.040434
...,...,...,...,...,...,...,...
258,25.0,25.422,0.932,-65,24.542434,0.422,-0.457566
259,25.0,25.832,0.279,-71,24.952434,0.832,-0.047566
260,25.0,25.618,0.280,-71,24.738434,0.618,-0.261566
261,25.0,25.774,0.305,-78,24.894434,0.774,-0.105566


In [387]:
raw_error= df['Error_before_offset']
calibrated_error=df['Error_after_offset']
print(np.mean(calibrated_error))
print(np.mean(raw_error))


from sklearn.metrics import mean_squared_error
from math import sqrt


mse_b = mean_squared_error(df['True_Range'], df['Est_Range'])
print("Mean Squared Error in meter: {:.3f}" .format(mse_b))

rmse_b=sqrt(mean_squared_error(df['True_Range'], df['Est_Range'])) 
print('Root Mean Squared Error (RMSE) on new data in meter: {:.3f}'.format(rmse_b))


mse_a = mean_squared_error(df['True_Range'], df['Final_range'])
print("Mean Squared Error in meter: {:.3f}" .format(mse_a))

rmse_a=sqrt(mean_squared_error(df['True_Range'], df['Final_range'])) 
print('Root Mean Squared Error (RMSE) on new data in meter: {:.3f}'.format(rmse_a))

-3.029445772620582e-16
0.8795658914728677
Mean Squared Error in meter: 0.874
Root Mean Squared Error (RMSE) on new data in meter: 0.935
Mean Squared Error in meter: 0.101
Root Mean Squared Error (RMSE) on new data in meter: 0.317


In [388]:
# # Specify the file name and path where you want to save the file
file_path = 'Offset_validate'+str(actual_dist)+'m.csv'


# Save the DataFrame to a CSV file using the to_csv() method
df.to_csv(file_path, index=False)

print(f'File saved to {file_path}!')

File saved to Offset_validate25m.csv!
